In [119]:
import pandas as pd
from scipy import spatial


In [120]:
PATH = r'/home/das432hz/sistemas_recomendacion_movies/datos/Ficheros_y_datasets'
genre_names = [
    'movie_id','unknown', 'Action', 'Adventure', 'Animation', 'Children\'s', 
    'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 
    'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 
    'War', 'Western', 'movie_name']
df_items = pd.read_csv(PATH+'/items.txt',sep="	", encoding='latin-1',header=None, names= genre_names)
df_genre = pd.read_csv(PATH+'/genre.txt',sep="	", encoding='latin-1',header=None)
df_calif = pd.read_csv(PATH+r'/u1_base.txt',sep="	", encoding='latin-1',header=None,names=["users","movie_id","rating"])
df_users = pd.read_csv(PATH+r'/users.txt',sep="	", encoding='latin-1',header=None, names=["users","age","gender","ocuppation"])

In [121]:
df_concat = df_users.merge(df_calif, left_on='users', right_on='users')
df_full = df_concat.merge(df_items, left_on='movie_id', right_on='movie_id')

In [123]:
#crear vector de preferencias
df_pref = pd.DataFrame([])
generos_lista = ['unknown','Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical',
       'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
df_pref[generos_lista] = df_full[generos_lista].mul(df_full['rating'], axis= 0 )
df_pref['users']  = df_full["users"]
df_pref = df_pref.groupby('users').mean()

In [124]:
df_full

,users,age,gender,ocuppation,movie_id,rating,unknown,Action,Adventure,Animation,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,movie_name
0,1,24,M,technician,1,5,0,0,0,1,...,0,0,0,0,0,0,0,0,0,Toy Story (1995)
1,1,24,M,technician,2,3,0,1,1,0,...,0,0,0,0,0,0,1,0,0,GoldenEye (1995)
2,1,24,M,technician,3,4,0,0,0,0,...,0,0,0,0,0,0,1,0,0,Four Rooms (1995)
3,1,24,M,technician,4,3,0,1,0,0,...,0,0,0,0,0,0,0,0,0,Get Shorty (1995)
4,1,24,M,technician,5,3,0,0,0,0,...,0,0,0,0,0,0,1,0,0,Copycat (1995)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79995,943,22,M,student,1067,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Bottle Rocket (1996)
79996,943,22,M,student,1074,4,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Reality Bites (1994)
79997,943,22,M,student,1188,3,0,1,0,0,...,0,0,0,0,0,0,0,0,1,Young Guns II (1990)
79998,943,22,M,student,1228,3,0,1,0,0,...,0,0,0,0,0,0,0,0,0,Under Siege 2: Dark Territory (1995)


In [138]:
def basado_contenido(df_full,df_items,df_pref):
    movies=[]
    for elements in range(len(df_pref)): 
        elements=elements+1
        movies_todelete=df_full[df_full['users']==elements]['movie_id'].to_list()
        movies_torecomend = df_items[~df_items['movie_id'].isin(movies_todelete)]
        best_genders_names=list(pd.DataFrame(df_pref.iloc[(elements-1),:].sort_values(ascending=False)).index)[0:5]
        best_genders_values=list(df_pref.iloc[(elements-1),:].sort_values(ascending=False))[0:5]
        best_genders = {best_genders_names[i]: best_genders_values[i] for i in range(len(best_genders_names))}
        for key,value in best_genders.items():
            movies_torecomend[str(key)]=movies_torecomend.loc[:,str(key)]*value
            print(key,value)
        data_drop=['movie_id', 'movie_name']
        movies_torecomend['Ratio'] = movies_torecomend.drop(data_drop, axis=1).sum(axis=1)
        movies_torecomend=movies_torecomend.sort_values(by='Ratio',ascending=False)
        movies_torecomend_names=list(movies_torecomend.loc[:,'movie_name'])[0:5]
        movies.append(movies_torecomend_names)

    return movies

In [141]:
movies[0]

["Kid in King Arthur's Court, A (1995)",
 'Transformers: The Movie, The (1986)',
 'Space Jam (1996)',
 'Hercules (1997)',
 'Army of Darkness (1993)']

In [139]:
movies=basado_contenido(df_full,df_items,df_pref)


/tmp/ipykernel_24137/1647799192.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_torecomend[str(key)]=movies_torecomend.loc[:,str(key)]*value
/tmp/ipykernel_24137/1647799192.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_torecomend['Ratio'] = movies_torecomend.drop(data_drop, axis=1).sum(axis=1)
/tmp/ipykernel_24137/1647799192.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See

Drama 1.651851851851852
Comedy 1.288888888888889
Action 0.7037037037037037
Romance 0.6814814814814815
Thriller 0.6592592592592592
Drama 2.275
Romance 1.325
Thriller 0.825
Comedy 0.825
Crime 0.6
Drama 1.3928571428571428
Thriller 0.7857142857142857
Comedy 0.7142857142857143
Action 0.6071428571428571
Sci-Fi 0.6071428571428571
Thriller 1.7142857142857142
Action 1.5714285714285714
Drama 1.0714285714285714
Crime 1.0
Mystery 0.9285714285714286
Comedy 1.3956043956043955
Action 1.1318681318681318
Sci-Fi 0.8241758241758241
Adventure 0.7582417582417582
Horror 0.42857142857142855
Drama 1.5454545454545454
Comedy 1.2727272727272727
Romance 0.7272727272727273
Action 0.4909090909090909
Adventure 0.42727272727272725
Drama 1.6150234741784038
Action 0.7746478873239436
Comedy 0.7652582159624414
Thriller 0.7511737089201878
Sci-Fi 0.5539906103286385
Action 2.9
Adventure 1.4666666666666666
Sci-Fi 1.3666666666666667
Thriller 1.1333333333333333
Crime 0.7333333333333333
Romance 2.3333333333333335
Comedy 1.83333

/tmp/ipykernel_24137/1647799192.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_torecomend[str(key)]=movies_torecomend.loc[:,str(key)]*value
/tmp/ipykernel_24137/1647799192.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_torecomend['Ratio'] = movies_torecomend.drop(data_drop, axis=1).sum(axis=1)
/tmp/ipykernel_24137/1647799192.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See

Drama 3.090909090909091
Thriller 1.7272727272727273
Romance 1.6363636363636365
Action 1.3636363636363635
War 1.0909090909090908
Drama 2.0
Comedy 0.6875
Mystery 0.625
Thriller 0.5
Crime 0.5
Drama 2.1818181818181817
Comedy 0.9696969696969697
Thriller 0.8787878787878788
War 0.48484848484848486
Crime 0.45454545454545453
Comedy 1.2242990654205608
Drama 0.9719626168224299
Children's 0.42990654205607476
Thriller 0.3925233644859813
Musical 0.38317757009345793
Drama 2.1538461538461537
Romance 1.1538461538461537
Crime 1.0
Mystery 0.6923076923076923
Thriller 0.46153846153846156
Action 2.0
Adventure 1.875
Romance 1.8125
Comedy 1.1875
Sci-Fi 1.125
Drama 2.4285714285714284
Comedy 0.7428571428571429
Sci-Fi 0.6571428571428571
Thriller 0.6571428571428571
Adventure 0.5428571428571428
Action 2.0
Thriller 1.25
Sci-Fi 1.1875
Drama 1.125
Adventure 1.0
Action 1.4
Thriller 1.3
Drama 1.0
Adventure 0.8666666666666667
Comedy 0.6666666666666666
Action 3.5
Adventure 2.0
War 1.6
Thriller 1.5
Sci-Fi 1.4
Action 1.478

/tmp/ipykernel_24137/1647799192.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_torecomend['Ratio'] = movies_torecomend.drop(data_drop, axis=1).sum(axis=1)
/tmp/ipykernel_24137/1647799192.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_torecomend[str(key)]=movies_torecomend.loc[:,str(key)]*value
/tmp/ipykernel_24137/1647799192.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See

...

/tmp/ipykernel_24137/1647799192.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_torecomend['Ratio'] = movies_torecomend.drop(data_drop, axis=1).sum(axis=1)
/tmp/ipykernel_24137/1647799192.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_torecomend[str(key)]=movies_torecomend.loc[:,str(key)]*value
/tmp/ipykernel_24137/1647799192.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See

Drama 1.8470149253731343
Comedy 1.376865671641791
Romance 1.037313432835821
Thriller 0.7686567164179104
Action 0.746268656716418
Drama 2.7209302325581395
Romance 1.0232558139534884
Comedy 0.5348837209302325
Thriller 0.46511627906976744
Action 0.37209302325581395
Comedy 1.4019607843137254
Drama 1.0784313725490196
Romance 0.8627450980392157
Children's 0.7843137254901961
Action 0.7352941176470589
Drama 1.3
Action 1.1291666666666667
Comedy 1.1166666666666667
Thriller 0.7958333333333333
Romance 0.5833333333333334
Comedy 1.3801169590643274
Drama 1.0175438596491229
Adventure 0.9532163742690059
Action 0.9473684210526315
Children's 0.9064327485380117
Romance 2.3
Comedy 2.15
Drama 1.95
War 0.85
Mystery 0.25
Drama 1.3312883435582823
Comedy 1.1993865030674846
Action 0.8803680981595092
Thriller 0.7607361963190185
Romance 0.6042944785276073
Drama 1.504201680672269
Comedy 0.8991596638655462
Action 0.8571428571428571
Sci-Fi 0.8151260504201681
War 0.6974789915966386
Drama 2.3617021276595747
Romance 1.2

/tmp/ipykernel_24137/1647799192.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_torecomend['Ratio'] = movies_torecomend.drop(data_drop, axis=1).sum(axis=1)
/tmp/ipykernel_24137/1647799192.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_torecomend[str(key)]=movies_torecomend.loc[:,str(key)]*value
/tmp/ipykernel_24137/1647799192.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See

Drama 1.6923076923076923
Thriller 1.2692307692307692
Action 0.5769230769230769
Romance 0.5769230769230769
Mystery 0.5769230769230769
Drama 1.5425867507886435
Comedy 0.9810725552050473
Action 0.8233438485804416
Romance 0.7066246056782335
Thriller 0.7003154574132492
Drama 2.142857142857143
Romance 1.1428571428571428
Comedy 1.1142857142857143
Thriller 0.5428571428571428
Crime 0.42857142857142855
Drama 1.912621359223301
Comedy 1.174757281553398
Romance 0.9223300970873787
War 0.3300970873786408
Action 0.23300970873786409
Drama 1.8110599078341014
Comedy 0.8986175115207373
Thriller 0.8064516129032258
Romance 0.7004608294930875
Action 0.5806451612903226
Drama 1.8846153846153846
Thriller 1.5384615384615385
Mystery 0.7692307692307693
Romance 0.7307692307692307
Crime 0.6153846153846154
Comedy 1.2272727272727273
Action 1.1181818181818182
Adventure 0.9727272727272728
Drama 0.9545454545454546
Romance 0.9090909090909091
Action 1.2677165354330708
Comedy 1.204724409448819
Adventure 0.8661417322834646
S

/tmp/ipykernel_24137/1647799192.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_torecomend[str(key)]=movies_torecomend.loc[:,str(key)]*value
/tmp/ipykernel_24137/1647799192.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_torecomend['Ratio'] = movies_torecomend.drop(data_drop, axis=1).sum(axis=1)
/tmp/ipykernel_24137/1647799192.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See

In [43]:
df_usuario = df_full[df_full['users']==1]
df_usuario.head(10)

,users,age,gender,ocuppation,movie_id,rating,unknown,Action,Adventure,Animation,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,movie_name
0,1,24,M,technician,1,5,0,0,0,1,...,0,0,0,0,0,0,0,0,0,Toy Story (1995)
1,1,24,M,technician,2,3,0,1,1,0,...,0,0,0,0,0,0,1,0,0,GoldenEye (1995)
2,1,24,M,technician,3,4,0,0,0,0,...,0,0,0,0,0,0,1,0,0,Four Rooms (1995)
3,1,24,M,technician,4,3,0,1,0,0,...,0,0,0,0,0,0,0,0,0,Get Shorty (1995)
4,1,24,M,technician,5,3,0,0,0,0,...,0,0,0,0,0,0,1,0,0,Copycat (1995)
5,1,24,M,technician,7,4,0,0,0,0,...,0,0,0,0,0,1,0,0,0,Twelve Monkeys (1995)
6,1,24,M,technician,8,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Babe (1995)
7,1,24,M,technician,9,5,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Dead Man Walking (1995)
8,1,24,M,technician,11,2,0,0,0,0,...,0,0,0,0,0,0,1,0,0,Seven (Se7en) (1995)
9,1,24,M,technician,13,5,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Mighty Aphrodite (1995)


In [44]:
movies_todelete=df_full[df_full['users']==1]['movie_id'].to_list()

In [45]:
movies_todelete

[1,
 2,
 3,
 4,
 5,
 7,
 8,
 9,
 11,
 13,
 15,
 16,
 18,
 19,
 21,
 22,
 25,
 26,
 28,
 29,
 30,
 32,
 34,
 35,
 37,
 38,
 40,
 41,
 42,
 43,
 45,
 46,
 48,
 50,
 52,
 55,
 57,
 58,
 59,
 63,
 66,
 68,
 71,
 75,
 77,
 79,
 83,
 87,
 88,
 89,
 93,
 94,
 95,
 99,
 101,
 105,
 106,
 109,
 110,
 111,
 115,
 116,
 119,
 122,
 123,
 124,
 126,
 127,
 131,
 133,
 135,
 136,
 137,
 138,
 139,
 141,
 142,
 144,
 146,
 147,
 149,
 152,
 153,
 156,
 158,
 162,
 165,
 166,
 167,
 168,
 169,
 172,
 173,
 176,
 178,
 179,
 181,
 182,
 187,
 191,
 192,
 194,
 195,
 197,
 198,
 199,
 203,
 204,
 205,
 207,
 211,
 216,
 217,
 220,
 223,
 231,
 234,
 237,
 238,
 239,
 240,
 244,
 245,
 246,
 247,
 249,
 251,
 256,
 257,
 261,
 263,
 268,
 269,
 270,
 271]

In [46]:
movies_torecomend = df_items[~df_items['movie_id'].isin(movies_todelete)]

In [47]:
movies_torecomend

,movie_id,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,movie_name
5,6,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,Shanghai Triad (Yao a yao yao dao waipo qiao) ...
9,10,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,Richard III (1995)
11,12,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,"Usual Suspects, The (1995)"
13,14,0,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,"Postino, Il (1994)"
16,17,0,1,0,0,0,1,1,0,0,...,0,1,0,0,0,0,1,0,0,From Dusk Till Dawn (1996)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,1678,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,Mat' i syn (1997)
1678,1679,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,B. Monkey (1998)
1679,1680,0,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,Sliding Doors (1998)
1680,1681,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,You So Crazy (1994)


In [48]:
import numpy as np 
best_genders_names=list(pd.DataFrame(df_pref.iloc[0,:].sort_values(ascending=False)).index)[0:5]
best_genders_values=list(df_pref.iloc[0,:].sort_values(ascending=False))[0:5]

best_genders = {best_genders_names[i]: best_genders_values[i] for i in range(len(best_genders_names))}

In [49]:
best_genders

{'Drama': 1.651851851851852,
 'Comedy': 1.288888888888889,
 'Action': 0.7037037037037037,
 'Romance': 0.6814814814814815,
 'Thriller': 0.6592592592592592}

In [50]:
for key,value in best_genders.items():
    movies_torecomend[str(key)]=movies_torecomend.loc[:,str(key)]*value
    print(key,value)

Drama 1.651851851851852
Comedy 1.288888888888889
Action 0.7037037037037037
Romance 0.6814814814814815
Thriller 0.6592592592592592


/tmp/ipykernel_24137/186234276.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_torecomend[str(key)]=movies_torecomend.loc[:,str(key)]*value


In [51]:
movies_torecomend

,movie_id,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,movie_name
5,6,0,0.000000,0,0,0,0.000000,0,0,1.651852,...,0,0,0,0,0.000000,0,0.000000,0,0,Shanghai Triad (Yao a yao yao dao waipo qiao) ...
9,10,0,0.000000,0,0,0,0.000000,0,0,1.651852,...,0,0,0,0,0.000000,0,0.000000,1,0,Richard III (1995)
11,12,0,0.000000,0,0,0,0.000000,1,0,0.000000,...,0,0,0,0,0.000000,0,0.659259,0,0,"Usual Suspects, The (1995)"
13,14,0,0.000000,0,0,0,0.000000,0,0,1.651852,...,0,0,0,0,0.681481,0,0.000000,0,0,"Postino, Il (1994)"
16,17,0,0.703704,0,0,0,1.288889,1,0,0.000000,...,0,1,0,0,0.000000,0,0.659259,0,0,From Dusk Till Dawn (1996)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,1678,0,0.000000,0,0,0,0.000000,0,0,1.651852,...,0,0,0,0,0.000000,0,0.000000,0,0,Mat' i syn (1997)
1678,1679,0,0.000000,0,0,0,0.000000,0,0,0.000000,...,0,0,0,0,0.681481,0,0.659259,0,0,B. Monkey (1998)
1679,1680,0,0.000000,0,0,0,0.000000,0,0,1.651852,...,0,0,0,0,0.681481,0,0.000000,0,0,Sliding Doors (1998)
1680,1681,0,0.000000,0,0,0,1.288889,0,0,0.000000,...,0,0,0,0,0.000000,0,0.000000,0,0,You So Crazy (1994)


In [54]:
# Lista de columnas a excluir de la suma
data_drop=['movie_id', 'movie_name']
# Suma por filas excluyendo columnas
movies_torecomend['Ratio'] = movies_torecomend.drop(data_drop, axis=1).sum(axis=1)


/tmp/ipykernel_24137/3285007141.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_torecomend['Ratio'] = movies_torecomend.drop(data_drop, axis=1).sum(axis=1)


In [57]:
movies_torecomend=movies_torecomend.sort_values(by='Ratio',ascending=False)

In [61]:
movies_torecomend_names=list(movies_torecomend.loc[:,'movie_name'])[0:5]

In [62]:
movies_torecomend_names

["Kid in King Arthur's Court, A (1995)",
 'Transformers: The Movie, The (1986)',
 'Space Jam (1996)',
 'Hercules (1997)',
 'Army of Darkness (1993)']

### sistema coolabora

In [20]:
df_pref.iloc[0]

unknown        0.000000
Action         0.703704
Adventure      0.311111
Animation      0.185185
Children's     0.237037
Comedy         1.288889
Crime          0.274074
Documentary    0.177778
Drama          1.651852
Fantasy        0.022222
Film-Noir      0.037037
Horror         0.096296
Musical        0.096296
Mystery        0.103704
Romance        0.681481
Sci-Fi         0.585185
Thriller       0.659259
War            0.318519
Western        0.029630
Name: 1, dtype: float64

In [12]:
# sistema por contenido compara un usuario contra todos los suarios
#hacer pearson
#regresar n mejores usuarios
# definir una formula para buscar afinidad entre las peliculas que aparezcan entre los usuarios.
# tendriamos que encontrar los n mejroes usuarios. Listar las peliculas de esos mejores nsuarios
#crear un ratio de afinidad para cada pelicula vista por los n usuarios mas cercanos tomando en cuenta el ratio de afinidad entre usuarios y la puntuacion 
movie_a = df_pref.iloc[0].to_list()
movie_b = df_pref.iloc[10].to_list()
spatial.distance.cosine(movie_a, movie_b)

0.0176321404827241

In [13]:
df_full[df_full['users']==1][['rating','movie_name']].sort_values(by='rating', ascending=False)

,rating,movie_name
11,5,French Twist (Gazon maudit) (1995)
133,5,Gattaca (1997)
0,5,Toy Story (1995)
132,5,"Full Monty, The (1997)"
90,5,"Wrong Trousers, The (1993)"
...,...,...
58,1,Operation Dumbo Drop (1995)
115,1,Batman Returns (1992)
23,1,Free Willy 2: The Adventure Home (1995)
14,1,Muppet Treasure Island (1996)


In [14]:
# ssitema colaborativo: comparo con la informacion de un usuario checo las peliculas que  mas le han gustado y le recomiendo con base en esa pelicula
df_full[df_full['users']==1][['rating','movie_name']].sort_values(by='rating', ascending=False)

,rating,movie_name
11,5,French Twist (Gazon maudit) (1995)
133,5,Gattaca (1997)
0,5,Toy Story (1995)
132,5,"Full Monty, The (1997)"
90,5,"Wrong Trousers, The (1993)"
...,...,...
58,1,Operation Dumbo Drop (1995)
115,1,Batman Returns (1992)
23,1,Free Willy 2: The Adventure Home (1995)
14,1,Muppet Treasure Island (1996)


### Colaborativo

pedir informacion  a un usuario nuevo, este tiene que ratear los generos. esta puntuacion crea un vector de preferencias para cada genero, tomar las n mejores calificaciones de cada genero y volver 0 el resto.

una vez que volvemos 0 el resto ver como vamos hacer la comparacion del vector de preferencias actual, con el resto
quitando las que ya vio 

In [15]:
df_full.groupby('movie_id')[generos_lista].mean()

,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movie_id,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1678,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1679,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1680,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [45]:
df_full.groupby('movie_id')[generos_lista].mean()
movie_a = df_pref.iloc[0].to_list()
movie_b = df_pref.iloc[10].to_list()
spatial.distance.cosine(movie_a, movie_b)

0.01763214048272399

In [29]:
df_full.columns

Index(['users', 'age', 'gender', 'ocuppation', 'movie_id', 'rating', 'unknown',
       'Action', 'Adventure', 'Animation', 'Children's', 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical',
       'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western',
       'movie_name'],
      dtype='object')

In [30]:
#sistema demografico, recomiendo a traves de grupos 
lista_preferencias = ['age', 'gender', 'ocuppation']
df_2 = df_full[lista_preferencias]

In [33]:
df_2_num = pd.get_dummies(df_2[lista_preferencias], prefix='category')


In [37]:
df_2_num

,age,category_F,category_M,category_administrator,category_artist,category_doctor,category_educator,category_engineer,category_entertainment,category_executive,...,category_marketing,category_none,category_other,category_programmer,category_retired,category_salesman,category_scientist,category_student,category_technician,category_writer
0,24,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,53,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,42,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,47,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,21,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79995,17,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
79996,17,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
79997,31,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
79998,28,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [31]:
df_2

,age,gender,ocuppation
0,24,M,technician
1,53,F,other
2,42,M,executive
3,47,M,educator
4,21,M,entertainment
...,...,...,...
79995,17,M,student
79996,17,M,student
79997,31,M,executive
79998,28,M,writer


In [ ]:
from lightgbm import LGBMClassifier


num_folds = 5
kf = KFold(n_splits = num_folds, random_state = 42)
error = 0
models = []
for i, (train_index, val_index) in enumerate(kf.split(train)):
    if i + 1 < num_folds:
        continue
    print(train_index.max(), val_index.min())
    train_X = train[train_index]
    val_X = train[val_index]
    train_y = target[train_index]
    val_y = target[val_index]
    lgb_train = lgb.Dataset(train_X, train_y > 0)
    lgb_eval = lgb.Dataset(val_X, val_y > 0)
    params = {
            'boosting_type': 'gbdt',
            'objective': 'regression',
            'metric': {'rmse'},
            'learning_rate': 0.1,
            'feature_fraction': 0.8,
            'bagging_fraction': 1.0,
            'bagging_freq' : 0
             
            }
    gbm_class = lgb.train(params,
                lgb_train,
                num_boost_round=2000,
                valid_sets=(lgb_train, lgb_eval),
               early_stopping_rounds=20,
               verbose_eval = 20)
    
    lgb_train = lgb.Dataset(train_X[train_y > 0], train_y[train_y > 0])
    lgb_eval = lgb.Dataset(val_X[val_y > 0] , val_y[val_y > 0])
    params = {
            'boosting_type': 'gbdt',
            'objective': 'regression',
            'metric': {'rmse'},
            'learning_rate': 0.1,
            'feature_fraction': 0.8,
            'bagging_fraction': 1.0,
            'bagging_freq' : 0
                         }
    gbm_regress = lgb.train(params,
                lgb_train,
                num_boost_round=2000,
                valid_sets=(lgb_train, lgb_eval),
               early_stopping_rounds=20,
               verbose_eval = 20)
#     models.append(gbm)

    y_pred = (gbm_class.predict(val_X, num_iteration=gbm_class.best_iteration) > .5) *\
    (gbm_regress.predict(val_X, num_iteration=gbm_regress.best_iteration))
    error += np.sqrt(mean_squared_error(y_pred, (val_y)))/num_folds
    print(np.sqrt(mean_squared_error(y_pred, (val_y))))
    break
print(error)

In [51]:
df_concatenado = pd.concat([df_users.set_index('users'), df_calif.set_index('users')], axis=1)
df_concateated = pd.concat([df_concatenado.set_index('movie_id'), df_items.set_index('movie_id')], axis=1)
# Si quieres resetear el índice después de la concatenación:
df_concatenado.reset_index(inplace=True)

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [45]:
# Specify the columns to encode
columns_to_encode = ['gender','ocuppation']

# Perform one-hot encoding on selected columns
df_encoded = pd.get_dummies(df_concatenado
, columns=columns_to_encode)

In [46]:
df_encoded

,level_0,index,users,age,movie_id,rating,gender_F,gender_M,ocuppation_administrator,ocuppation_artist,...,ocuppation_marketing,ocuppation_none,ocuppation_other,ocuppation_programmer,ocuppation_retired,ocuppation_salesman,ocuppation_scientist,ocuppation_student,ocuppation_technician,ocuppation_writer
0,0,0,1,24,1,5,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,1,1,24,2,3,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
2,2,2,1,24,3,4,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
3,3,3,1,24,4,3,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
4,4,4,1,24,5,3,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79995,79995,79995,943,22,1067,2,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
79996,79996,79996,943,22,1074,4,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
79997,79997,79997,943,22,1188,3,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
79998,79998,79998,943,22,1228,3,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
